In [1]:
from IPython.display import display
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
if  not os.path.abspath('../../dashgrid') in sys.path:
    sys.path.append(os.path.abspath('../../dashgrid'))
if  not os.path.abspath('../../dashgrid/dashgrid') in sys.path:
    sys.path.append(os.path.abspath('../../dashgrid/dashgrid'))

import datetime
import pandas as pd
import numpy as np
from dashgrid import dgrid_components as dgc
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)


In [2]:
class PlotlyCandles():
    BAR_WIDTH=.5
    def __init__(self,df,title='candle plot',number_of_ticks_display=20):
        self.df = df.copy()
        #  and make sure the first index is 1 NOT 0!!!!!!
        self.df.index = np.array(list(range(len(df))))+1
        
        self.title = title
        self.number_of_ticks_display = number_of_ticks_display
        
    def get_candle_shapes(self):
        df = self.df.copy()
        xvals = df.index.values #chg    
        lows = df.low.values
        highs = df.high.values
        closes = df.close.values
        opens = df.open.values
        df['is_red'] = df.open>=df.close
        is_reds = df.is_red.values
        lines_below_box = [{
                    'type': 'line',
                    'x0': xvals[i],
                    'y0': lows[i],
                    'x1': xvals[i],
                    'y1': closes[i] if is_reds[i] else opens[i],
                    'line': {
                        'color': 'rgb(55, 128, 191)',
                        'width': 1.5,
                    }
                } for i in range(len(xvals))
        ]

        lines_above_box = [{
                    'type': 'line',
                    'x0': xvals[i],
                    'y0': opens[i] if is_reds[i] else closes[i],
                    'x1': xvals[i],
                    'y1': highs[i],
                    'line': {
                        'color': 'rgb(55, 128, 191)',
                        'width': 1.5,
                    }
                }for i in range(len(xvals))
        ]


        boxes = [{
                    'type': 'rect',
                    'xref': 'x',
                    'yref': 'y',
                    'x0': xvals[i]- PlotlyCandles.BAR_WIDTH/2,
                    'y0': closes[i] if is_reds[i] else opens[i],
                    'x1': xvals[i]+ PlotlyCandles.BAR_WIDTH/2,
                    'y1': opens[i] if is_reds[i] else closes[i],
                    'line': {
                        'color': 'rgb(55, 128, 191)',
                        'width': 1,
                    },
                    'fillcolor': 'rgba(255, 0, 0, 0.6)' if is_reds[i] else 'rgba(0, 204, 0, 0.6)',
                } for i in range(len(xvals))
        ]
        shapes = lines_below_box + boxes + lines_above_box
        return shapes

    
    def get_figure(self):
        '''
        Use Plotly to create a financial candlestick chart.
        The DataFrame df_in must have columns called:
         'date','open','high','low','close'
        '''
        # Step 0: get important constructor values (so you don't type 'self.'' too many times)
        df_in = self.df.copy()
        title=self.title
        number_of_ticks_display=self.number_of_ticks_display
        
        # Step 1: only get the relevant columns and sort by date
        cols_to_keep = ['date','open','high','low','close','volume']
        df = df_in[cols_to_keep].sort_values('date')
        # Step 2: create a data frame for "green body" days and "red body" days
        # Step 3: create the candle shapes that surround the scatter plot in trace1
        shapes = self.get_candle_shapes()

        # Step 4: create an array of x values that you want to show on the xaxis
        spaces = len(df)//number_of_ticks_display
        indices = list(df.index.values[::spaces]) + [max(df.index.values)]
        tdvals = df.loc[indices].date.values

        # Step 5: create a layout
        layout1 = go.Layout(
            showlegend=False,
            title = title,
            margin = dict(t=100),
            xaxis = go.layout.XAxis(
                tickmode = 'array',
                tickvals = indices,
                ticktext = tdvals,
                tickangle=90,
                showgrid = True,
                showticklabels=True,
                anchor='y2', 
            ),       
            yaxis1 = go.layout.YAxis(
                range =  [min(df.low.values), max(df.high.values)],
                domain=[.22,1]
            ),
            yaxis2 = go.layout.YAxis(
                range =  [0, max(df.volume.values)],
                domain=[0,.2]
            ),
            shapes = shapes
        )

        # Step 6: create a scatter object, and put it into an array
        def __hover_text(r):
            d = r.date
            o = r.open
            h = r.high
            l = r.low
            c = r.close
            v = r.volume
            t = f'date: {d}<br>open: {o}<br>high: {h}<br>low: {l}<br>close: {c}<br>volume: {v}' 
            return t
        df['hover_text'] = df.apply(__hover_text,axis=1)
        hover_text = df.hover_text.values

        # Step 7: create scatter (close values) trace.  The candle shapes will surround the scatter trace
        trace1 = go.Scatter(
            x=df.index.values,
            y=df.close.values,
            mode = 'markers',
            text = hover_text,
            hoverinfo = 'text',
            xaxis='x',
            yaxis='y1'
        )

        # Step 8: create the bar trace (volume values)
        trace2 = go.Bar(
            x=df.index.values,
            y=df.volume.values,
            width = PlotlyCandles.BAR_WIDTH,
            xaxis='x',
            yaxis='y2'
        )

        # Step 9: create the final figure and pass it back to the caller
        fig1 = {'data':[trace1,trace2],'layout':layout1}
        return fig1
    
    def plot(self):
        fig = self.get_figure()
        iplot(fig)
        return fig

In [4]:
all_days=1000
end_date = datetime.datetime.now()
beg_date = end_date - datetime.timedelta(all_days)
date_series = pd.bdate_range(beg_date,end_date)
trade_dates = date_series.astype(str).str.replace('-','').astype(int)
n = len(trade_dates)
changes = np.random.lognormal(0,.15/256**.5,n-1)
initial = np.array([100.0])
closes = np.cumprod(np.append(initial,changes)).round(2)
open_ranges = np.random.lognormal(0,.3/256**.5,n)
opens = (closes * open_ranges).round(2)
low_ranges = np.random.lognormal(.1,.2/256**.5,n)
lows = np.array([min(x,y) for x,y in zip(opens,closes)]) - low_ranges
lows = lows.round(2)
high_ranges = np.random.lognormal(.1,.2/256**.5,n)
highs = np.array([max(x,y) for x,y in zip(opens,closes)]) + high_ranges
highs = highs.round(2)

volume_changes = np.random.lognormal(0,.15/256**.5,n-1)
initial_volume = np.array([1000000.0]) * np.random.lognormal(0,.15/256**.5)
volumes = np.cumprod(np.append(initial_volume,volume_changes)).round(0)


df_pseudo = pd.DataFrame({'date':trade_dates,'open':opens,'high':highs,'low':lows,
                  'close':closes,'volume':volumes})


In [91]:
fig1 = PlotlyCandles(df_pseudo.iloc[-50:],number_of_ticks_display=20,title='Random Walk')
iplot(fig1.get_figure())

In [21]:
?dgc.FigureComponent

In [125]:
def dash_app_1():
    days_to_show = 50
    df_to_show = df_pseudo.iloc[-days_to_show:]
    
    dt1_comp = dgc.DashTableComponent(
        'dt1',df_to_show,None,title='Random Walk',editable_columns=['close']
    )
    def _gr_lambda(input_list):
        if input_list is None or len(input_list)<1 or input_list[0] is None:
            return [None]
        df = pd.DataFrame(input_list)
        p = PlotlyCandles(df,number_of_ticks_display=20,title='Random Walk').get_figure()
        return p
    
    gr1_comp = dgc.FigureComponent('f1',dt1_comp,_gr_lambda)

    app = dash.Dash()
    comp_list = [dt1_comp,gr1_comp]
    html_list = [c.html for c in comp_list]
    app.layout = dgc.html.Div(html_list)
    comp_list[0].callback(app)
    comp_list[1].callback(app)
    return app


In [ ]:
DASH_APP_TO_RUN = dash_app_1
app = DASH_APP_TO_RUN()
app.run_server(host='127.0.0.1',port=8500)

dt.columns [{'name': 'close', 'id': 'close', 'editable': True}, {'name': 'date', 'id': 'date', 'editable': False}, {'name': 'high', 'id': 'high', 'editable': False}, {'name': 'low', 'id': 'low', 'editable': False}, {'name': 'open', 'id': 'open', 'editable': False}, {'name': 'volume', 'id': 'volume', 'editable': False}]
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2019-07-22 14:08:11,604 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2019-07-22 14:08:16,179 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 14:08:16] "GET / HTTP/1.1" 200 -
2019-07-22 14:08:16,236 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 14:08:16] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-22 14:08:16,253 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 14:08:16] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-22 14:08:16,260 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 14:08:16] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
2019-07-22 14:08:16,278 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 14:08:16] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563025508 HTTP/1.1" 200 -
2019-07-22 14:08:16,285 - werkzeug - INFO - 127.0.0.1 

dt.columns [{'name': 'close', 'id': 'close', 'editable': True}, {'name': 'date', 'id': 'date', 'editable': False}, {'name': 'high', 'id': 'high', 'editable': False}, {'name': 'low', 'id': 'low', 'editable': False}, {'name': 'open', 'id': 'open', 'editable': False}, {'name': 'volume', 'id': 'volume', 'editable': False}]
_gr_lambda figure = {'data': [Scatter({
    'hoverinfo': 'text',
    'mode': 'markers',
    'text': array(['date: 20190514.0<br>open: 99.89<br>high: 103.72<br>low: 98.79<br>close: 102.63<br>volume: 636552.0',
                   'date: 20190515.0<br>open: 100.55<br>high: 102.71<br>low: 99.44<br>close: 101.63<br>volume: 646547.0',
                   'date: 20190516.0<br>open: 101.17<br>high: 102.26<br>low: 99.92<br>close: 101.01<br>volume: 648562.0',
                   'date: 20190517.0<br>open: 102.15<br>high: 103.24<br>low: 100.37<br>close: 101.48<br>volume: 652567.0',
                   'date: 20190520.0<br>open: 103.46<br>high: 104.56<br>low: 101.37<br>close: 102.48<br

2019-07-22 14:08:17,754 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 14:08:17] "POST /_dash-update-component HTTP/1.1" 200 -
2019-07-22 14:08:17,824 - werkzeug - INFO - 127.0.0.1 - - [22/Jul/2019 14:08:17] "POST /_dash-update-component HTTP/1.1" 200 -


_gr_lambda figure = {'data': [Scatter({
    'hoverinfo': 'text',
    'mode': 'markers',
    'text': array(['date: 20190514.0<br>open: 99.89<br>high: 103.72<br>low: 98.79<br>close: 102.63<br>volume: 636552.0',
                   'date: 20190515.0<br>open: 100.55<br>high: 102.71<br>low: 99.44<br>close: 101.63<br>volume: 646547.0',
                   'date: 20190516.0<br>open: 101.17<br>high: 102.26<br>low: 99.92<br>close: 101.01<br>volume: 648562.0',
                   'date: 20190517.0<br>open: 102.15<br>high: 103.24<br>low: 100.37<br>close: 101.48<br>volume: 652567.0',
                   'date: 20190520.0<br>open: 103.46<br>high: 104.56<br>low: 101.37<br>close: 102.48<br>volume: 655911.0',
                   'date: 20190521.0<br>open: 103.3<br>high: 104.38<br>low: 99.49<br>close: 100.6<br>volume: 638676.0',
                   'date: 20190522.0<br>open: 100.25<br>high: 102.11<br>low: 99.15<br>close: 101.01<br>volume: 640622.0',
                   'date: 20190523.0<br>open: 99.24<br>high

In [111]:
l = dgc.flatten_layout(app)
for c in l:
    print('***********************************************')
    print(c)

***********************************************
DataTable(columns=[{'name': 'close', 'id': 'close', 'editable': True}, {'name': 'date', 'id': 'date', 'editable': False}, {'name': 'high', 'id': 'high', 'editable': False}, {'name': 'low', 'id': 'low', 'editable': False}, {'name': 'open', 'id': 'open', 'editable': False}, {'name': 'volume', 'id': 'volume', 'editable': False}], css=[{'selector': 'table', 'rule': 'width: 100%;'}], data=[{'close': 102.63, 'date': 20190514.0, 'high': 103.72, 'low': 98.79, 'open': 99.89, 'volume': 636552.0}, {'close': 101.63, 'date': 20190515.0, 'high': 102.71, 'low': 99.44, 'open': 100.55, 'volume': 646547.0}, {'close': 101.01, 'date': 20190516.0, 'high': 102.26, 'low': 99.92, 'open': 101.17, 'volume': 648562.0}, {'close': 101.48, 'date': 20190517.0, 'high': 103.24, 'low': 100.37, 'open': 102.15, 'volume': 652567.0}, {'close': 102.48, 'date': 20190520.0, 'high': 104.56, 'low': 101.37, 'open': 103.46, 'volume': 655911.0}, {'close': 100.6, 'date': 20190521.0, '

In [122]:
import importlib
importlib.reload(dgc)

<module 'dashgrid.dgrid_components' from '/Users/bperlman1/Documents/billybyte/pyliverisk/dashgrid/dashgrid/dgrid_components.py'>

In [59]:
x = True
[1,3,4] + ([4,5,6] if x else [])

[1, 3, 4, 4, 5, 6]